In [29]:
from binance.client import Client
from binance.enums import *
import pandas as pd
import pandas_ta as ta
from datetime import datetime, timedelta

api_key = ''
api_secret = ''

print('logged in')
client = Client(api_key, api_secret)

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%d-%m-%y")
    d2 = datetime.strptime(d2, "%d-%m-%y")
    return abs((d2 - d1).days)

def stochastic(df):
    df['L14'] = df['low'].rolling(window=9).min()
    df['H14'] = df['high'].rolling(window=9).max()
    df['%K'] = 100*((df['close'] - df['L14']) / (df['H14'] - df['L14'])).rolling(window=3).mean()
    df['%D'] = df['%K'].rolling(window=3).mean()
    return df

price_list = []
day_list = []

quantity = 1000

while True :
    # Get data 1 Minute
    columns = ['opentime','open','high','low','close','Volume','Close time','Quote asset volume','Number of trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore']
    minutes = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1MINUTE, "30 minute ago UTC")
    df_minutes = pd.DataFrame(minutes, columns=columns)
    df_minutes.drop(columns=['Volume','Close time','Quote asset volume','Number of trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore'],inplace=True)
    df_minutes['opentime'] = df_minutes['opentime'].apply(lambda x : datetime.fromtimestamp(x/1000))
    df_minutes['close'] = df_minutes['close'].astype(float)
    df_minutes['change_7'] = df_minutes['close'].pct_change(7)
    df_minutes['buy'] = df_minutes['close'] * 1.015
    df_minutes['day'] = df_minutes['opentime'].dt.strftime('%d-%m-%y')

    # Get data 1 Hour
    hour = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "1 day ago UTC")
    df_hour = pd.DataFrame(hour, columns=columns)
    df_hour.drop(columns=['Volume','Close time','Quote asset volume','Number of trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore'],inplace=True)
    df_hour['opentime'] = df_hour['opentime'].apply(lambda x : datetime.fromtimestamp(x/1000))
    df_hour['close'] = df_hour['close'].astype(float)
    df_hour['change_7'] = df_hour['close'].pct_change(7)
    df_hour['buy'] = df_hour['close'] * 1.015
    df_hour['day'] = df_hour['opentime'].dt.strftime('%d-%m-%y')
    df_hour_w_stoch = stochastic(df_hour)
    
    # มีการซื้อขายก่อนหน้านี้
    if len(day_list) > 0 :

        # กรณีที่ซื้อขายไปเรียบร้อยและเกินมา 3 วัน พร้อมที่จะซื้อใหม่ -> ซื้อ
        if (df_minutes.change_7.iloc[-2] < -0.07) & (days_between(day_list[-1],df_minutes['day'].iloc[-2]) > 3) :
            # reset day_list
            day_list = []
            # Buy 
            order = client.create_order('BTCUSDT',
                                        side=SIDE_BUY,
                                        type=ORDER_TYPE_LIMIT,
                                        quantity=quantity,
                                        price=df_minutes['buy'].iloc[-2])

            # Collect Date & Price
            price_list.append(df_minutes['buy'].iloc[-2])
            day_list.append(df_minutes['day'].iloc[-2])
        
        # กรณีซื้อขายในวันเดียวกันและมีราคาแตกต่างกันเกิน 5% -> ซื้อ
        elif (df_minutes.change_7.iloc[-2] < -0.07) & (((price_list[-1] - df_minutes['close'].iloc[-2])/price_list[-1]) > 0.05) & ((day_list[-1] == df_minutes['day'].iloc[-2])) :
            # Buy
            order = client.create_order('BTCUSDT',
                                        side=SIDE_BUY,
                                        type=ORDER_TYPE_LIMIT,
                                        quantity=quantity,
                                        price=df_minutes['buy'].iloc[-2])

            # Collect Date & Price
            price_list.append(df_minutes['buy'].iloc[-2])
            day_list.append(df_minutes['day'].iloc[-2])

        # เข้าเงื่อนไขการขาย แท่งที่ 3 %K>80 และ %D>80 และแทงที่ 2 %D>%K -> ขาย 
        elif ((df_hour_w_stoch['%K'].iloc[-3] > 80) & (df_hour_w_stoch['%D'].iloc[-3] > 80) & (df_hour_w_stoch['%D'].iloc[-2] > df_hour_w_stoch['%K'].iloc[-2])) :

            # Sell 
            order = client.create_order('BTCUSDT',
                                        side=SIDE_SELL,
                                        type=ORDER_TYPE_LIMIT,
                                        quantity=float(client.get_account()['balances'][0]['free']),
                                        price=df_minutes['close'].iloc[-1])
            # Reset Price_list
            price_list = []

    # เริ่มต้นการซื้อขายครั้งแรก
    else :
        if (df_minutes.change_7.iloc[-2] < -0.07) :
            # Buy
            order = client.create_order('BTCUSDT',
                                        side=SIDE_BUY,
                                        type=ORDER_TYPE_LIMIT,
                                        quantity=quantity,
                                        price=df_minutes['buy'].iloc[-2])

            # Collect Date & Price
            price_list.append(df_minutes['buy'].iloc[-2])
            day_list.append(df_minutes['day'].iloc[-2])
        else:
            pass #print(df_minutes.change_7.iloc[-2])

logged in
-0.006795311594539943
-0.006795311594539943


KeyboardInterrupt: 

In [ ]:
while True:
    if df_minutes_btc['close'].iloc[-2] < 3600 :
        if condition2 :
            order = client.create_order('BTCUSDT',
                                        side=SIDE_BUY,
                                        type=ORDER_TYPE_LIMIT,
                                        quantity=1000,
                                        price=3500)

    if df_minutes_luna['close'].iloc[-2] < 3600 :
        order = client.create_order('BTCUSDT',
                                    side=SIDE_BUY,
                                    type=ORDER_TYPE_LIMIT,
                                    quantity=1000,
                                    price=3500)
    

    if df_minutes_doge['close'].iloc[-2] < 3600 :
        order = client.create_order('BTCUSDT',
                                    side=SIDE_BUY,
                                    type=ORDER_TYPE_LIMIT,
                                    quantity=1000,
                                    price=3500)
        break